In [1]:
import os

import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, utils

In [2]:
Lx = 470
Ly = 470
Lz = 60
Rp = 6
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.SurfaceMarkers()
phases = commons.Phases()
CELL_TYPES = commons.CellTypes()
scale = (800/470) * 1e-6
scale_factor = [scale, scale, scale]
outdir = "mesh/laminate/"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'laminate.msh')
tetr_path = os.path.join(outdir, 'tetr.xdmf')
tria_path = os.path.join(outdir, 'tria.xdmf')

## Build Laminate Geometry of Cylindrical SE Strands

In [3]:
df = 470 * pd.read_csv('data/laminate.csv')
df.head()

,x,y,z
0,160.839113,449.497240,NaN
1,8.388525,73.296053,NaN
2,363.830211,436.152462,NaN
3,352.934738,356.118427,NaN
4,89.346537,267.904715,NaN


In [47]:
gmsh.initialize()
gmsh.model.add('laminate')
box_se = gmsh.model.occ.addBox(0, 0, 0, Lx, Ly, Lz)
gmsh.model.occ.synchronize()
box_am = gmsh.model.occ.addBox(0, 0, Lz, Lx, Ly, Rp)
gmsh.model.occ.synchronize()
cylinders = []
for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if (x + Rp >= 470) or (y + Rp >= 470):
        continue
    if (x - Rp <= 0) or (y - Rp <= 0):
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, 0, 0, 0, Lz, Rp)
    cylinders.append(cyl)
    gmsh.model.occ.synchronize()

se_phase = gmsh.model.occ.cut([(3, box_se)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
union = gmsh.model.occ.fuse([(3, box_am)], [(3, c) for c in cylinders])
gmsh.model.occ.synchronize()
vols = gmsh.model.occ.getEntities(3)
se_vol = gmsh.model.addPhysicalGroup(3, [vols[0][1]], phases.electrolyte)
am_vol = gmsh.model.addPhysicalGroup(3, [vols[1][1]], phases.active_material)
gmsh.model.occ.synchronize()
right = []
left = []
insulated = []
interface = []
for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    if np.isclose(com[2], Lz + Rp):
        right.append(surf[1])
    elif np.isclose(com[2], Lz + 0.5 * Rp):
        if np.isclose(com[1], 235) or np.isclose(com[0], 235):
            interface.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], 0):
        left.append(surf[1])
    else:
        interface.append(surf[1])
gmsh.model.addPhysicalGroup(2, left, markers.left_cc)
gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.addPhysicalGroup(2, interface, markers.am_se_interface)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

[(3, 1), (3, 2)]
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 25 (Circle)
Info    : [ 10%] Meshing curve 26 (Line)
Info    : [ 10%] Meshing curve 27 (Circle)
Info    : [ 10%] Meshing curve 28 (Circle)
Info    : [ 10%] Meshing curve 29 (Line)
Info    : [ 10%] Meshing curve 30 (Circle)
Info    : [ 10%] Meshing curve 31 (Circle)
Info    : [ 10%] Meshing curve 32 (Line)
Info    : [ 10%] Meshing curve 33 (Circle)
Info    : [ 10%] Meshing curve 34 (Circle)
Info    : [ 10%] Meshing curve 35 (Line)
Info    : [ 10%] Meshing curve 36 (Circle)
Info    : [ 10%] Meshing curve 37 (Circle)
Info    : [ 10%] Meshing curve 38 (Line)
Info    : [ 10%] Meshing curve 39 (Circle)
Info    : [ 10%] Meshing curve 40 (Circle)
Info    : [ 10%] Meshing curve 41 (Line)
Info    : [ 10%] Meshing curve 42 (Circle)
Info    : [ 10%] Meshing curve 43 (Circle)
Info    : [ 10%] Meshing curve 44 (Line)
Info    : [ 10%] Meshing curve 45 (Circle)
Info    : [ 10%] Meshing curve 46 (Circle)
Info    : [ 10%] Meshing cu

Info    : [ 20%] Meshing surface 136 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 139 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 142 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 145 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 148 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 151 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 154 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 157 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 160 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 163 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 166 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 169 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 172 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 175 (Cylinder, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 178 (Cylinder, Frontal-Delaunay)
Info    : 

Info    : Done tetrahedrizing 18606 nodes (Wall 0.161271s, CPU 0.159508s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    :  - Added 54 Steiner points
Info    : Done reconstructing mesh (Wall 0.654197s, CPU 0.651365s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 358.597 (nodes removed 0 0)
Info    : It. 500 - 16 nodes created - worst tet radius 24.8772 (nodes removed 164 320)
Info    : It. 1000 - 47 nodes created - worst tet radius 4.09741 (nodes removed 507 446)
Info    : It. 1500 - 115 nodes created - worst tet radius 3.57205 (nodes removed 844 541)
Info    : It. 2000 - 174 nodes created - worst tet radius 3.27732 (nodes removed 1186 640)
Info    : It. 2500 - 253 nodes created - worst tet radius 2.98365 (nodes removed 1543 704)
Info    : It. 3000 - 328 nodes created - worst tet radius 2.77447 (nodes removed 1860 812)
Info    : It. 3500 - 393 nodes crea

Info    : Optimization starts (volume = 1.2796e+07) with worst = 7.47904e-10 / average = 0.619542:
Info    : 0.00 < quality < 0.10 :      2098 elements
Info    : 0.10 < quality < 0.20 :      2234 elements
Info    : 0.20 < quality < 0.30 :      3675 elements
Info    : 0.30 < quality < 0.40 :      6165 elements
Info    : 0.40 < quality < 0.50 :      9141 elements
Info    : 0.50 < quality < 0.60 :     17608 elements
Info    : 0.60 < quality < 0.70 :     16563 elements
Info    : 0.70 < quality < 0.80 :     19396 elements
Info    : 0.80 < quality < 0.90 :     13177 elements
Info    : 0.90 < quality < 1.00 :      5958 elements
Info    : 4831 edge swaps, 934 node relocations (volume = 1.2796e+07): worst = 7.47904e-10 / average = 0.635986 (Wall 0.160059s, CPU 0.159914s)
Info    : 5898 edge swaps, 1574 node relocations (volume = 1.2796e+07): worst = 7.47904e-10 / average = 0.636492 (Wall 0.275343s, CPU 0.275241s)
Info    : 0.00 < quality < 0.10 :       551 elements
Info    : 0.10 < quality < 0.

Info    : 1552 edge swaps, 72 node relocations (volume = 1.78651e+06): worst = 0.0014269 / average = 0.230228 (Wall 0.242282s, CPU 0.242307s)
Info    : 1916 edge swaps, 123 node relocations (volume = 1.78651e+06): worst = 0.0014269 / average = 0.230083 (Wall 0.477087s, CPU 0.477327s)
Info    : No ill-shaped tets in the mesh :-)
Info    : 0.00 < quality < 0.10 :     13310 elements
Info    : 0.10 < quality < 0.20 :      2613 elements
Info    : 0.20 < quality < 0.30 :      1950 elements
Info    : 0.30 < quality < 0.40 :      2006 elements
Info    : 0.40 < quality < 0.50 :      1621 elements
Info    : 0.50 < quality < 0.60 :      1038 elements
Info    : 0.60 < quality < 0.70 :       587 elements
Info    : 0.70 < quality < 0.80 :       695 elements
Info    : 0.80 < quality < 0.90 :      1379 elements
Info    : 0.90 < quality < 1.00 :       471 elements
Info    : Done optimizing mesh (Wall 0.947054s, CPU 0.946921s)
Info    : 36541 nodes 183344 elements
Info    : Writing 'mesh/laminate/lamina

Info    : Done writing 'mesh/laminate/laminate.msh'


In [48]:
msh = meshio.read(msh_path)
tetr_unscaled = geometry.create_mesh(msh, CELL_TYPES.tetra)
tetr_unscaled.write(tetr_path)
tetr_scaled = geometry.scale_mesh(tetr_unscaled, CELL_TYPES.tetra, scale_factor=scale_factor)
tetr_scaled.write(tetr_path)
tria_unscaled = geometry.create_mesh(msh, CELL_TYPES.triangle)
tria_unscaled.write(tria_path)
tria_scaled = geometry.scale_mesh(tria_unscaled, CELL_TYPES.triangle, scale_factor=scale_factor)
tria_scaled.write(tria_path)